In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import clean_up as cln 



In [2]:
# import "Annual_Parking_Study_Data" to creat 'pkdt' dataframe
# read data
col_names = ['Date Time','Unitdesc','Side', 'Parking_Spaces', 'Total_Vehicle_Count', 'Construction', 'Event Closure']
pkdt = cln.read_data(r'.\raw_data\Annual_Parking_Study_Data.csv', col_names)
pkdt = pkdt.dropna()



C:\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [3]:
#filter the data with 'Construction' and 'Event' column are "No"
values = ['Yes', 'yes', 'ERROR: #N/A']
pkdt = cln.data_filter(pkdt, 'Construction', values)
pkdt = cln.data_filter(pkdt, 'Event Closure', ['Yes'])

pkdt = pkdt.drop('Construction', axis = 1)
pkdt = pkdt.drop('Event Closure', axis = 1)



In [4]:
#group rows with same 'Time' and 'Unitdesc', for other columns, find the sum
pkdt = pkdt.groupby(['Date Time', 'Unitdesc'], as_index = False)['Parking_Spaces', 'Total_Vehicle_Count'].sum()



In [5]:
#find the occupancy and append it to the pkdt
Occupancy = np.minimum(pkdt['Total_Vehicle_Count'] / pkdt['Parking_Spaces'], 1)
pkdt['Occupancy'] = Occupancy
pkdt = pkdt.dropna()
pkdt = pkdt.drop('Total_Vehicle_Count', axis = 1)



C:\Miniconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in minimum
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
#retrivev 'hour' column from general datetime
pkdt['Date Time'] = pd.to_datetime(pkdt['Date Time'], format = '%m-%d-%y %H:%M:%S')
times = pd.DatetimeIndex(pkdt['Date Time'])
pkdt['Hour'] = times.hour

#for 'Hour' column, replace 0 with 24
pkdt = pkdt.replace({'Hour': 0}, 24)



In [7]:
#group data by 'hour' and 'Unidesc' and average other columns
pkdt = pkdt.groupby(['Hour', 'Unitdesc'], as_index = False)['Parking_Spaces', 'Occupancy'].mean()



In [8]:
#group data by 'hour' and 'Unidesc' and average other columns
pkdt = pkdt.groupby(['Hour', 'Unitdesc'], as_index = False)['Parking_Spaces', 'Occupancy'].mean()



In [9]:
pkdt


,Hour,Unitdesc,Parking_Spaces,Occupancy
0,1,10TH AVE BETWEEN E PIKE ST AND E PINE ST,40.000000,0.350000
1,1,10TH AVE BETWEEN E UNION ST AND E PIKE ST,20.000000,0.550000
2,1,11TH AVE BETWEEN E PIKE ST AND E PINE ST,29.000000,0.586207
3,1,12TH AVE BETWEEN E MADISON ST AND E PIKE ST,18.000000,0.388889
4,1,12TH AVE BETWEEN E PIKE ST AND E PINE ST,3.000000,1.000000
5,1,13TH AVE BETWEEN E MADISON ST AND E PIKE ST,5.000000,1.000000
6,1,13TH AVE BETWEEN E PIKE ST AND E PINE ST,12.000000,0.500000
7,1,14TH AVE BETWEEN E PIKE ST AND E PINE ST,9.000000,0.444444
8,1,15TH AVE BETWEEN E MADISON ST AND E PINE ST,4.000000,1.000000
9,1,BELMONT AVE BETWEEN E PIKE ST AND E PINE ST,8.000000,1.000000


In [10]:
pkdt.to_csv('Occupancy_per_hour.csv')